### Integration WP + PS

In [32]:
import pandas as pd
import numpy as np

t_amb = pd.read_csv('WP_40599_2014.csv')
t_amb.set_index('time', inplace=True)
t_amb.index = pd.to_datetime(t_amb.index)

# Pufferspeicher auswählen 
# Vaillant https://www.vaillant.de/heizung/produkte/pufferspeicher-359745.html#specification
# VPS R 100/1 M - Bereitschaftswärmeverlust Speicher: 0,81 kWh/24h
# VPS R 200/1 B - Bereitschaftswärmeverlust Speicher: 1,4 kWh/24h
# VIH RW 300 - Bereitschaftswärmeverlust Speicher: 1,52 kWh/24
# TWL Pufferspeicher PR 500 Liter - Bereitschaftswärmeverlust Speicher: 1,4 kWh/24h -- https://www.solardirekt24.de/TWL-Pufferspeicher-PR-500-Liter-OEkoLine-A-Isolierung-PR.0500.Iso-A
VPS_100 = 100
VPS_200 = 200
VIH_300 = 300
TWL_PR_500 = 500

V_sp_einfach = 20*9

if V_sp_einfach > VPS_200:
    if V_sp_einfach <= VIH_300:
        V_ps = VIH_300
        V_verlust = 1.52 # kWh/24h
    else:
        V_ps = TWL_PR_500
        V_verlust = 1.4 # kWh/24h
elif V_sp_einfach <= VPS_200:
    if V_sp_einfach <= VPS_100:
        V_ps = VPS_100
        V_verlust = 0.81 # kWh/24h
    else:
        V_ps= VPS_200
        V_verlust = 1.4 # kWh/24h

print('Volumen PS [l]: ', V_ps)


# Wärmegehalt Pufferspeicher
dichte = 1 # kg/m^3
c_wasser = 4.18 # kJ/(kg·K)
T_n_vor = 55
T_n_rueck = 45
Q_ps = round(V_ps*dichte*c_wasser*(T_n_vor - T_n_rueck)/3600, 3)
print('Wärmegehalt PS [kWh]:',  Q_ps) # kWh


# WP und PS Zusammenfügen
t_amb['Wärmegehalt PS'] = np.nan
t_amb['Ladezustand PS'] = np.nan
t_amb['Heizleistung neu'] = np.nan

# Set 1. Reihe 
t_amb.iloc[0, t_amb.columns.get_loc('Wärmegehalt PS')] = Q_ps  
t_amb.iloc[0, t_amb.columns.get_loc('Ladezustand PS')] = 1 
t_amb.iloc[0, t_amb.columns.get_loc('Heizleistung neu')] = t_amb.iloc[0, t_amb.columns.get_loc('Heizleistung')]  

# Schleife über den Index des DataFrames (ab der ersten Zeile, Index 0)
for time in t_amb.index[1:]:  # ab der zweiten Zeile
    # Berechnung der Heizleistung basierend auf dem Ladezustand und dem Wärmebedarf
    if time.month > 3 and time.month < 10:  # Zeit von April bis Oktober
        t_amb.loc[time, 'Heizleistung neu'] = 0
    elif t_amb.loc[time - pd.Timedelta(hours=1), 'Wärmegehalt PS'] > t_amb.loc[time, 'Wärmebedarf']:
        t_amb.loc[time, 'Heizleistung neu'] = 0
    elif t_amb.loc[time - pd.Timedelta(hours=1), 'Ladezustand PS'] < 0.5:
        if Q_ps - t_amb.loc[time - pd.Timedelta(hours=1), 'Wärmegehalt PS'] > t_amb.loc[time, 'Heizleistung']:
            t_amb.loc[time, 'Heizleistung neu'] = t_amb.loc[time, 'Heizleistung'] + Q_ps - t_amb.loc[time - pd.Timedelta(hours=1), 'Wärmegehalt PS']
        else:
            t_amb.loc[time, 'Heizleistung neu'] = t_amb.loc[time, 'Heizleistung']
    
    # Berechnungen für den Wärmegehalt und Ladezustand werden erst jetzt durchgeführt
    if time != t_amb.index[0]:  # Für alle Zeitstempel ab Index 1
        # Berechnung des neuen Wärmegehalts basierend auf dem Wert der vorherigen Stunde
        t_amb.loc[time, 'Wärmegehalt PS'] = t_amb.loc[time - pd.Timedelta(hours=1), 'Wärmegehalt PS'] - t_amb.loc[time, 'Wärmebedarf'] - V_verlust / 24 - t_amb.loc[time, 'Heizleistung neu']
    
    # Berechnung des Ladezustands
    t_amb.loc[time, 'Ladezustand PS'] = t_amb.loc[time, 'Wärmegehalt PS'] / Q_ps


Volumen PS [l]:  200
Wärmegehalt PS [kWh]: 2.322


C:\Users\DianaEspinosaLozano\AppData\Local\Temp\ipykernel_25744\3951511606.py:74: RuntimeWarning: overflow encountered in scalar subtract
  t_amb.loc[time, 'Wärmegehalt PS'] = t_amb.loc[time - pd.Timedelta(hours=1), 'Wärmegehalt PS'] - t_amb.loc[time, 'Wärmebedarf'] - V_verlust / 24 - t_amb.loc[time, 'Heizleistung neu']


In [33]:
t_amb

,temp,Lastverhältnis,e^x,T_vor,T_rueck,COP,Heizleistung,Wärmebedarf,Wärmegehalt PS,Ladezustand PS,Heizleistung neu
time,,,,,,,,,,,
2014-01-01 00:00:00,6.3,0.492806,1.260006,43.881684,38.953627,3.23,3.434211,0.518056,2.322000,1.000000,3.434211
2014-01-01 01:00:00,5.0,0.539568,1.274443,45.056198,39.660515,2.90,3.947368,0.823876,1.439790,0.620065,0.000000
2014-01-01 02:00:00,5.1,0.535971,1.273348,44.967379,39.607667,2.90,3.907895,1.116919,0.264538,0.113927,0.000000
2014-01-01 03:00:00,5.5,0.521583,1.268943,44.609629,39.393802,3.02,3.750000,1.359407,-4.903202,-2.111629,3.750000
2014-01-01 04:00:00,5.7,0.514388,1.266725,44.429232,39.285347,3.02,3.671053,1.280667,-17.138457,-7.380903,10.896255
...,...,...,...,...,...,...,...,...,...,...,...
2014-12-31 19:00:00,1.9,0.651079,1.307264,47.700356,41.189565,2.47,5.171053,4.807082,-inf,-inf,inf
2014-12-31 20:00:00,1.4,0.669065,1.312377,48.109164,41.418517,2.40,5.368421,3.853547,-inf,-inf,inf
2014-12-31 21:00:00,1.1,0.679856,1.315423,48.352353,41.553792,2.34,5.486842,1.934805,-inf,-inf,inf
